In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json, re
from pathlib import Path
from bs4 import BeautifulSoup
from IPython.display import display
import requests

2026-01-29 20:57:17,680 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# -------- CONFIG TURNO --------
TURNO_SELECIONADO = 1  # 1 = primeiro turno, 2 = segundo turno (padrao)

if TURNO_SELECIONADO == 1:
    TURNO_INICIO = 1
    TURNO_FIM = 19
    TURNO_OFFSET = 0
    TOTAL_RODADAS = 19
else:
    TURNO_INICIO = 20
    TURNO_FIM = 38
    TURNO_OFFSET = 19
    TOTAL_RODADAS = 19

def rodada_ui_para_cartola(ui):
    return ui + TURNO_OFFSET

def rodada_cartola_para_ui(cartola):
    return cartola - TURNO_OFFSET

UI_INICIO = 1
UI_FIM = 19


In [3]:
pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [4]:
# 👉 ajuste o caminho do HTML salvo
HTML_PATH = Path("pagina_liga_pontos_corridos.html")

# carrega html
if not HTML_PATH.exists():
    print(f"?? HTML n?o encontrado: {HTML_PATH} (pulando leitura)")
    html = ""
else:
    html = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")


import unicodedata

def clean_name(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00a0", " " )
    s = re.sub(r"[ \t\r\n\f\v]+", " ", s).strip()
    s = re.sub(r"\bMedonho\s*['\u2019\u00b4\u0301]\s*s\b", "Medonho\u00b4s", s)
    return s

# 1) seletor principal dessa página
nomes = [clean_name(el.get_text(" ", strip=True)) for el in soup.select("p.card-confrontos-pontos-corridos__time, p.card-participantes-pontos-corridos__time")]

# 2) fallbacks (caso salve outra variante da página)
if not nomes:
    nomes = [clean_name(el.get_text(" ", strip=True)) for el in soup.select("p.nome-time")]
if not nomes:
    nomes = [clean_name((a.get("title") or a.get_text(" ", strip=True)))
             for a in soup.select("a.link-perfil-time") if (a.get("title") or a.get_text(strip=True))]
if not nomes:
    nomes = [clean_name((img.get("title") or "").replace("Time ", ""))
               for img in soup.select("img[title^=\"Time \"]") if img.get("title")]


# 3) fallback final via regex (se o HTML veio “achatado”)
if not nomes:
    for m in re.finditer(r'class="card-confrontos-pontos-corridos__time"\s*>\s*([^<]+)<', html, re.I):
        nomes.append(clean_name(m.group(1)))

# dedup + ordena alfabética
visto, times = set(), []
for n in nomes:
    n = clean_name(n)
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML (se quiser usar no site)
OUT_JS = HTML_PATH.with_name("participantes_Liga_pontos_corridos.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n",
                  encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")


20 times encontrados
['Arran Katoko FC', 'bugredasmissões', 'C R Juvenal', 'Doug Leal F.C', 'Esquadrão Gazembrino', 'FBC Colorado', 'GaúchoDaFronteira F.C', 'GE Bebum', 'GrioTeam', 'Grêmio_Campeão_LA_27', 'JV5 Tricolor Gaúcho', 'La Primeira Patada Es Nuestra', 'lsauer fc', 'Medonho´s F.C.', 'NHU PORÃ SAF.', 'Pepe Leal FC', 'Pontaç0 F.C.', 'SC 100 Sono', 'SC ÉoINTER!', 'Texas Club 2026']
Snippet salvo em: participantes_Liga_pontos_corridos.js


In [5]:
JS_PATH = Path("participantes_Liga_pontos_corridos.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times

20 times


['Arran Katoko FC',
 'bugredasmissões',
 'C R Juvenal',
 'Doug Leal F.C',
 'Esquadrão Gazembrino',
 'FBC Colorado',
 'GaúchoDaFronteira F.C',
 'GE Bebum',
 'GrioTeam',
 'Grêmio_Campeão_LA_27',
 'JV5 Tricolor Gaúcho',
 'La Primeira Patada Es Nuestra',
 'lsauer fc',
 'Medonho´s F.C.',
 'NHU PORÃ SAF.',
 'Pepe Leal FC',
 'Pontaç0 F.C.',
 'SC 100 Sono',
 'SC ÉoINTER!',
 'Texas Club 2026']

In [6]:
# -------- GERAR TIMES.XLSX --------
from pathlib import Path
import pandas as pd

ARQ_TIMES = Path("times.xlsx")  # mesma pasta do notebook
ABA = "Times"

# Fonte: use a lista final gerada na celula anterior (preferencia: `times`)
if "times" in globals() and isinstance(times, list) and len(times) > 0:
    lista_nomes = times
elif "nomes_times" in globals() and isinstance(nomes_times, list) and len(nomes_times) > 0:
    lista_nomes = nomes_times
else:
    print("?? Nao encontrei a lista de times. Pulando esta etapa.")
    lista_nomes = []

lista_nomes = [str(x).strip() for x in lista_nomes if str(x).strip()]
lista_nomes = sorted(set(lista_nomes), key=str.casefold)

df_out = pd.DataFrame({"Nome": lista_nomes})

with pd.ExcelWriter(ARQ_TIMES, engine="openpyxl") as writer:
    df_out.to_excel(writer, sheet_name=ABA, index=False)

print(f"? times.xlsx criado/atualizado em: {ARQ_TIMES.resolve()} | Times: {len(df_out)}")
display(df_out.head())


? times.xlsx criado/atualizado em: C:\Users\ferna\Projetos\GitHub\cartola_2026\pontos_corridos\datasets_pontos_corridos\times.xlsx | Times: 20


,Nome
0,Arran Katoko FC
1,bugredasmissões
2,C R Juvenal
3,Doug Leal F.C
4,Esquadrão Gazembrino


In [7]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,Arran Katoko FC
1,bugredasmissões
2,C R Juvenal
3,Doug Leal F.C
4,Esquadrão Gazembrino


### Buscar IDs dos times no Cartola

In [8]:
# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 20
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [9]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_pontos_corridos.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_pontos_corridos.xlsx


,Nome do Time,ID do Time,Link do Time
0,Arran Katoko FC,19833277,https://cartola.globo.com/#!/time/19833277
1,bugredasmissões,19209079,https://cartola.globo.com/#!/time/19209079
2,C R Juvenal,1488983,https://cartola.globo.com/#!/time/1488983
3,Doug Leal F.C,287965,https://cartola.globo.com/#!/time/287965
4,Esquadrão Gazembrino,2916559,https://cartola.globo.com/#!/time/2916559
5,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
6,GaúchoDaFronteira F.C,2371918,https://cartola.globo.com/#!/time/2371918
7,GE Bebum,16411206,https://cartola.globo.com/#!/time/16411206
8,GrioTeam,14933455,https://cartola.globo.com/#!/time/14933455
9,Grêmio_Campeão_LA_27,47775950,https://cartola.globo.com/#!/time/47775950


### Gerar o dicionário ID -> Nome do Time

In [10]:
# Gerar o dicionário ID -> Nome do Time
if isinstance(df_urls, pd.DataFrame) and {"ID do Time", "Nome do Time"}.issubset(df_urls.columns):
    nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))
else:
    print("?? df_urls sem colunas esperadas (ID/Nome). Pulando mapeamento.")
    nomes_por_id = {}


# Mostrar parte do dicionário
# dict(list(nomes_por_id.items())[:5])
dict(list(nomes_por_id.items()))

{19833277: 'Arran Katoko FC',
 19209079: 'bugredasmissões',
 1488983: 'C R Juvenal',
 287965: 'Doug Leal F.C',
 2916559: 'Esquadrão Gazembrino',
 186283: 'FBC Colorado',
 2371918: 'GaúchoDaFronteira F.C',
 16411206: 'GE Bebum',
 14933455: 'GrioTeam',
 47775950: 'Grêmio_Campeão_LA_27',
 1747619: 'JV5 Tricolor Gaúcho',
 32966: 'La Primeira Patada Es Nuestra',
 44810918: 'lsauer fc',
 1867254: 'Medonho´s F.C.',
 4088673: 'NHU PORÃ SAF.',
 1326835: 'Pepe Leal FC',
 20651178: 'Pontaç0 F.C.',
 14709358: 'SC 100 Sono',
 184499: 'SC ÉoINTER!',
 1273719: 'Texas Club 2026'}

In [11]:
# ===============================
#  LIGA (TURNO SELECIONADO) COM PARCIAIS DA RODADA EM ANDAMENTO
#  -> Agora também funciona após o fim do campeonato,
#     carregando TODAS as rodadas do turno
# ===============================
import time
import json
import requests
import pandas as pd

# -------- CONFIG --------
CAP_MULT       = 1.5     # multiplicador do capitão nas parciais
SLEEP_REQ      = 0.9     # pausa anti-rate-limit
MAX_RETRIES    = 3

# -------- CartolaFCError (fallback) --------
try:
    from cartolafc import CartolaFCError as _CartolaErr
except Exception:
    _CartolaErr = Exception

# -------- Sessão HTTP (para endpoints públicos) --------
sess = requests.Session()
sess.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
})

def http_status_e_rodada():
    """Retorna (status_mercado, rodada_http) via endpoint público."""
    r = sess.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    d = r.json()
    return int(d.get("status_mercado", 0)), int(d.get("rodada_atual", 0))

def get_parciais():
    r = sess.get("https://api.cartola.globo.com/atletas/pontuados", timeout=30)
    r.raise_for_status()
    return r.json().get("atletas", {})

def fetch_pontuados(timeout=15):
    url = "https://api.cartola.globo.com/atletas/pontuados"
    try:
        r = sess.get(url, timeout=timeout)
        if r.status_code != 200:
            return {}
        data = r.json()
        atletas = data.get("atletas", {}) if isinstance(data, dict) else {}
        if not isinstance(atletas, dict):
            return {}
        out = {}
        for k, v in atletas.items():
            try:
                atleta_id = int(k)
                pont = v.get("pontuacao") if isinstance(v, dict) else None
                if pont is None:
                    continue
                out[atleta_id] = float(pont)
            except Exception:
                continue
        return out
    except Exception:
        return {}


def fetch_time_payload(time_id: int, rodada: int, timeout=15):
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartolafc.globo.com/time/{time_id}/{rodada}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = sess.get(url, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break
                data = r.json()
                if not isinstance(data, dict):
                    data = {}
                return data
            except Exception:
                time.sleep(0.5)
                continue
    return {}


def fetch_partidas_rodada(rodada: int, timeout=15):
    url = f"https://api.cartolafc.globo.com/partidas/{rodada}"
    try:
        r = sess.get(url, timeout=timeout)
        if r.status_code != 200:
            return []
        data = r.json()
        if isinstance(data, dict):
            partidas = data.get("partidas", [])
            return partidas if isinstance(partidas, list) else []
    except Exception:
        pass
    return []


def clubes_que_ja_jogaram(rodada: int):
    partidas = fetch_partidas_rodada(rodada)
    clubes = set()
    for p in partidas:
        if not isinstance(p, dict):
            continue
        status = (p.get("status_transmissao_tr") or p.get("status_transmissao") or "").upper()
        encerrada = status in ("ENCERRADA", "FINALIZADA")
        if p.get("placar_oficial_mandante") is not None or p.get("placar_oficial_visitante") is not None:
            encerrada = True
        if encerrada:
            mid = p.get("clube_casa_id")
            vid = p.get("clube_visitante_id")
            try:
                if mid is not None:
                    clubes.add(int(mid))
            except Exception:
                pass
            try:
                if vid is not None:
                    clubes.add(int(vid))
            except Exception:
                pass
    return clubes


def setor_por_posicao(posicao_id: int) -> str:
    mapa = {
        1: "Goleiro",
        2: "Laterais",
        3: "Zagueiros",
        4: "Meias",
        5: "Atacantes",
        6: "Tecnico",
    }
    return mapa.get(int(posicao_id), "")


def _id_int(val):
    try:
        return int(val)
    except Exception:
        return None


def calcular_parcial_time(time_id: int, rodada: int, mapa_pontuados: dict, clubes_jogaram: set):
    data = fetch_time_payload(time_id, rodada)
    atletas = data.get("atletas") if isinstance(data, dict) else None
    if not isinstance(atletas, list):
        return None

    titulares = []
    tecnico = None
    for a in atletas:
        if not isinstance(a, dict):
            continue
        pos = a.get("posicao_id")
        if pos == 6:
            tecnico = a
        else:
            titulares.append(a)

    luxo_id = data.get("reserva_luxo_id") if isinstance(data, dict) else None
    if luxo_id is not None:
        luxo_id = _id_int(luxo_id)

    # Capitao (1.5x)
    capitao_id = None
    try:
        capitao_id = data.get("capitao_id") if isinstance(data, dict) else None
        if capitao_id is None and isinstance(data.get("time"), dict):
            capitao_id = data["time"].get("capitao_id")
        capitao_id = _id_int(capitao_id)
    except Exception:
        capitao_id = None

    titulares_por_pos = {}
    for a in titulares:
        pos = a.get("posicao_id")
        titulares_por_pos.setdefault(pos, []).append(a)

    atletas_em_jogo = list(titulares)

    reservas = data.get("reservas", []) if isinstance(data, dict) else []
    if isinstance(reservas, list):
        for r in reservas:
            if not isinstance(r, dict):
                continue
            # reserva de luxo nao entra como banco
            if luxo_id is not None and _id_int(r.get("atleta_id")) == luxo_id:
                continue
            # banco so entra se reserva tiver pontuacao parcial positiva
            rid = _id_int(r.get("atleta_id"))
            if rid is None or rid not in mapa_pontuados:
                continue
            if float(mapa_pontuados.get(rid, 0.0)) <= 0.0:
                continue
            pos = r.get("posicao_id")
            if pos is None:
                continue
            candidatos = titulares_por_pos.get(pos, [])
            titular_sub = None
            for t in candidatos:
                tid = _id_int(t.get("atleta_id"))
                if tid is None:
                    continue
                clube_tid = _id_int(t.get("clube_id"))
                if tid not in mapa_pontuados and (clube_tid in clubes_jogaram):
                    titular_sub = t
                    break
            if titular_sub is None:
                continue
            try:
                atletas_em_jogo.remove(titular_sub)
            except ValueError:
                pass
            atletas_em_jogo.append(r)

    # Reserva de luxo
    luxo_capitao = False
    if luxo_id is not None:
        ids_em_jogo = set()
        for a in atletas_em_jogo:
            ids_em_jogo.add(_id_int(a.get("atleta_id")))
        if luxo_id not in ids_em_jogo:
            luxo_obj = None
            if isinstance(reservas, list):
                for r in reservas:
                    if isinstance(r, dict) and _id_int(r.get("atleta_id")) == luxo_id:
                        luxo_obj = r
                        break
            if isinstance(luxo_obj, dict):
                p_luxo = float(mapa_pontuados.get(luxo_id, 0.0))
                if p_luxo > 0:
                    setor_luxo = setor_por_posicao(luxo_obj.get("posicao_id"))
                    candidatos_setor = []
                    for a in atletas_em_jogo:
                        if setor_por_posicao(a.get("posicao_id")) == setor_luxo:
                            candidatos_setor.append(a)
                    if candidatos_setor:
                        # luxo so entra se TODOS do setor ja jogaram
                        todos_setor_jogaram = True
                        for c in candidatos_setor:
                            clube_c = _id_int(c.get("clube_id"))
                            if clube_c not in clubes_jogaram:
                                todos_setor_jogaram = False
                                break
                        if todos_setor_jogaram:
                            def pts(a):
                                try:
                                    return float(mapa_pontuados.get(int(a.get("atleta_id")), 0.0))
                                except Exception:
                                    return 0.0
                            pior_pts = None
                            piores = []
                            for c in candidatos_setor:
                                v = pts(c)
                                if pior_pts is None or v < pior_pts:
                                    pior_pts = v
                                    piores = [c]
                                elif v == pior_pts:
                                    piores.append(c)
                            pior = None
                            if len(piores) > 1 and capitao_id is not None:
                                for c in piores:
                                    if _id_int(c.get("atleta_id")) == capitao_id:
                                        pior = c
                                        break
                            if pior is None:
                                pior = piores[0] if piores else None
                            if pior is not None and p_luxo > pts(pior):
                                if capitao_id is not None and _id_int(pior.get("atleta_id")) == capitao_id:
                                    luxo_capitao = True
                                try:
                                    atletas_em_jogo.remove(pior)
                                except ValueError:
                                    pass
                                atletas_em_jogo.append(luxo_obj)

    total = 0.0
    for a in atletas_em_jogo:
        try:
            aid = int(a.get("atleta_id"))
        except Exception:
            continue
        total += float(mapa_pontuados.get(aid, 0.0))

    # Bonus do capitao (50%)
    if capitao_id is not None:
        cap_pts = float(mapa_pontuados.get(capitao_id, 0.0))
        if cap_pts != 0.0:
            ids_em_jogo = set()
            for a in atletas_em_jogo:
                ids_em_jogo.add(_id_int(a.get("atleta_id")))
            if capitao_id in ids_em_jogo:
                total += cap_pts * 0.5
            elif luxo_capitao and luxo_id is not None and luxo_id in ids_em_jogo:
                luxo_pts = float(mapa_pontuados.get(luxo_id, 0.0))
                if luxo_pts != 0.0:
                    total += luxo_pts * 0.5

    if isinstance(tecnico, dict):
        try:
            tid = int(tecnico.get("atleta_id"))
            total += float(mapa_pontuados.get(tid, 0.0))
        except Exception:
            pass

    return round(total, 2)

# -------- Regras de começo --------
def campeonato_comecou(api, ids_times):
    """
    Considera que o turno selecionado comecou se:
      - mercado indica rodada_atual >= TURNO_INICIO, e
      - ha pelo menos uma pontuacao nao-nula na rodada TURNO_INICIO para algum time.

    Se houver parciais disponiveis, assume que a rodada em andamento ja comecou.
    """
    try:
        rodada_atual = getattr(api.mercado(), "rodada_atual", None)
    except _CartolaErr:
        rodada_atual = TURNO_FIM + 1  # jogo acabou

    if rodada_atual is None or rodada_atual < TURNO_INICIO:
        return False

    # Se houver parciais, considera que ja comecou
    try:
        if fetch_pontuados():
            return True
    except Exception:
        pass

    for time_id in ids_times.values():
        try:
            p = api.time(time_id=time_id, rodada=TURNO_INICIO).ultima_pontuacao
            if p is not None:
                return True
        except _CartolaErr:
            # time sem escalacao nao invalida o inicio do turno
            continue
    return False

    for time_id in ids_times.values():
        try:
            p = api.time(time_id=time_id, rodada=TURNO_INICIO).ultima_pontuacao
            if p is not None:
                return True
        except _CartolaErr:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """
    Busca apenas rodadas concluídas do turno selecionado.

    Lógica:
    - Em temporada em andamento:
        * se rodada_atual = 26, rodadas concluídas = 20..25
    - Em temporada encerrada (forçamos rodada_atual = TURNO_FIM+1 = 20):
        * rodadas concluídas = 1..19
    """
    pontuacoes = {}
    # fim_exclusivo (rodada_atual tratada como "rodada em andamento")
    # concluídas = TURNO_INICIO .. (fim_exclusivo - 1)
    fim_exclusivo = min(max(rodada_atual, TURNO_INICIO), TURNO_FIM + 1)
    for rodada in range(TURNO_INICIO, fim_exclusivo):
        try:
            time_r = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_r.ultima_pontuacao
        except _CartolaErr as e:
            print(f"⚠️ Erro ao acessar rodada {rodada} (time {time_id}): {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    # 1) Colunas do turno
    colunas_turno = [f"Rodada {i}" for i in range(TURNO_INICIO, TURNO_FIM + 1)]

    # 2) Rodada atual via lib + status via HTTP
    try:
        rodada_atual_api = api.mercado().rodada_atual
    except _CartolaErr:
        # Se não conseguimos pegar (ex: game_over),
        # consideramos que NÃO há rodada em andamento -> todas concluídas.
        rodada_atual_api = TURNO_FIM + 1  # 20

    status_http, rodada_http = http_status_e_rodada()
    # rodada_ref é a rodada que consideramos "em andamento" para parciais
    rodada_ref = rodada_http if rodada_http else rodada_atual_api

    print(
        f"Status HTTP={status_http} | rodada_http={rodada_http} | "
        f"rodada_api={rodada_atual_api} | usando rodada_ref={rodada_ref}"
    )

    # 3) Se turno ainda não começou, placeholders 0
    if not campeonato_comecou(api, ids_times):
        print("📌 Turno selecionado ainda não começou. Criando estrutura com placeholders (0).")
        return pd.DataFrame(0, index=list(ids_times.keys()), columns=colunas_turno)

    # 4) Monta DF com rodadas ENCERRADAS (sem parciais ainda)
    dados = {}
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, int(time_id), rodada_atual_api)
        s = pd.Series({f"Rodada {k}": v for k, v in pontuacoes.items()})
        dados[nome] = s

    df = pd.DataFrame.from_dict(dados, orient="index")
    df = df.reindex(columns=colunas_turno)              # mantém ausentes como NaN
    df = df.apply(pd.to_numeric, errors="coerce")       # mantém numérico, preservando NaN

    # --- SANITY CHECK: só 1..19 mesmo ---
    colunas_ok = [f"Rodada {i}" for i in range(TURNO_INICIO, TURNO_FIM + 1)]
    colunas_ruins = [
        c for c in df.columns
        if c.startswith("Rodada ") and int(c.split()[1]) < TURNO_INICIO
    ]
    if colunas_ruins:
        df = df.drop(columns=colunas_ruins, errors="ignore")
    df = df.reindex(columns=colunas_ok)

    # 5) Injeta PARCIAIS na rodada em andamento (se houver pontuados e dentro do turno)
    if TURNO_INICIO <= rodada_ref <= TURNO_FIM:
        col_atual = f"Rodada {rodada_ref}"
        print(f"🟡 Rodada {rodada_ref} em andamento ? aplicando parciais?")
        parciais_map = fetch_pontuados()
        if not parciais_map:
            print("⚠️ Endpoint de parciais vazio no momento. Mantendo valores anteriores.")
        else:
            clubes_jogaram = clubes_que_ja_jogaram(rodada_ref)
            atualizados = 0
            for nome_time, time_id in ids_times.items():
                try:
                    total = calcular_parcial_time(int(time_id), rodada_ref, parciais_map, clubes_jogaram)
                    if total is None:
                        continue
                    df.loc[nome_time, col_atual] = round(total, 2)
                    atualizados += 1
                    time.sleep(SLEEP_REQ)
                except Exception:
                    continue
            print(f"✅ Times atualizados com parciais: {atualizados}/{len(ids_times)} na coluna '{col_atual}'")
    else:
        print("ℹ️ Sem rodada em andamento para parciais (rodada_ref fora do turno).")
# 6) (Opcional) Linha 'Lider_Rodada' quando houver alguma coluna com dado > 0
    try:
        cols_com_dado = [c for c in colunas_turno if df[c].notna().any()]
        if cols_com_dado and df[cols_com_dado].sum(numeric_only=True).sum() != 0:
            df.loc['Lider_Rodada'] = df[cols_com_dado].idxmax(axis=0)
    except Exception as e:
        print("⚠️ Não foi possível recalcular 'Lider_Rodada':", e)

    return df

# === Exemplo de uso ===
# Aqui assumimos que você já tem:
#   - api = cartolafc.Api(...)
#   - ids_times = {"Nome do Time 1": 123456, "Nome do Time 2": 789012, ...}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

# Visualização rápida
try:
    display(df_pontuacoes.T)
except Exception:
    print(df_pontuacoes.T.head(20))

# (Opcional) salvar em Excel
df_pontuacoes.to_excel("Pontuacoes_Liga_1_Turno_Completa.xlsx")
print("✅ Excel salvo: Pontuacoes_Liga_1_Turno_Completa.xlsx")




Status HTTP=2 | rodada_http=1 | rodada_api=1 | usando rodada_ref=1
🟡 Rodada 1 em andamento ? aplicando parciais?


✅ Times atualizados com parciais: 19/20 na coluna 'Rodada 1'


,Arran Katoko FC,bugredasmissões,C R Juvenal,Doug Leal F.C,Esquadrão Gazembrino,FBC Colorado,GaúchoDaFronteira F.C,GE Bebum,GrioTeam,Grêmio_Campeão_LA_27,JV5 Tricolor Gaúcho,La Primeira Patada Es Nuestra,lsauer fc,Medonho´s F.C.,Pepe Leal FC,Pontaç0 F.C.,SC 100 Sono,SC ÉoINTER!,Texas Club 2026,Lider_Rodada
Rodada 1,56.69,62.76,42.06,62.07,83.20,49.36,67.66,44.65,57.26,54.05,43.56,43.96,58.26,55.66,18.49,56.86,67.16,45.59,63.76,Esquadrão Gazembrino
Rodada 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


✅ Excel salvo: Pontuacoes_Liga_1_Turno_Completa.xlsx


### Função para definir a classificação dos times

In [12]:
def classificacao_por_grupo(
    df_rodadas: pd.DataFrame,
    df_pontuacoes: pd.DataFrame,
    rodadas_encerradas: set[int] | None = None,
    rodada_parcial: int | None = None,
):
    """
    Gera classificação por grupo a partir de confrontos (df_rodadas) e pontuações (df_pontuacoes).

    - `rodadas_encerradas`: rodadas 100% fechadas (contam normalmente).
    - `rodada_parcial`: rodada em andamento; só conta confrontos se ambos tiverem número e
      pelo menos UM deles > 0 (evita empates 0x0 de placeholder).
    """

    # --- 0) Normalizações para casar chaves ---
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').copy()
    if df_pontuacoes_times.index.dtype != "object":
        df_pontuacoes_times.index = df_pontuacoes_times.index.astype(str)
    df_pontuacoes_times.index = df_pontuacoes_times.index.str.strip()
    df_pontuacoes_times.columns = [str(c).strip() for c in df_pontuacoes_times.columns]

    def _norm_time(s: str) -> str:
        try:
            return clean_name(str(s)).casefold()
        except Exception:
            return str(s).strip().casefold()

    def _norm_key(s: str) -> str:
        try:
            base = _norm_time(s)
        except Exception:
            base = str(s).strip().casefold()
        return re.sub(r"[^a-z0-9]", "", base)

    idx_map = { _norm_key(i): i for i in df_pontuacoes_times.index }
    col_map = { str(c).strip(): c for c in df_pontuacoes_times.columns }
    col_team_map = { _norm_key(c): c for c in df_pontuacoes_times.columns }
    idx_row_map = { str(i).strip(): i for i in df_pontuacoes_times.index }



    required_cols = {"Grupo", "Mandante_Nome", "Visitante_Nome", "Rodada"}
    faltando = required_cols - set(df_rodadas.columns)
    if faltando:
        raise ValueError(f"df_rodadas está sem as colunas: {sorted(faltando)}")

    # --- 1) Pré-inicializa times (zerados) por grupo, a partir dos confrontos ---
    estatisticas: dict[str, dict[str, dict]] = {}
    for _, r in df_rodadas.iterrows():
        grupo = str(r["Grupo"]).strip()
        mandante = str(r["Mandante_Nome"]).strip()
        visitante = str(r["Visitante_Nome"]).strip()
        if grupo not in estatisticas:
            estatisticas[grupo] = {}
        for time in (mandante, visitante):
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0,
                    "Vitórias": 0,
                    "Empates": 0,
                    "Derrotas": 0,
                    "Total_Cartola": 0.0,
                    "Cartola_Sofrido": 0.0,
                }

    # helper seguro para ler pontuação do DF; None => não considerar
    def safe_pontuacao(nome_time: str, coluna_rodada: str):
        nome_key = _norm_key(nome_time)
        # orientacao A: times no index, rodadas nas colunas
        if (nome_key in idx_map) and (coluna_rodada in col_map):
            val = df_pontuacoes_times.at[idx_map[nome_key], col_map[coluna_rodada]]
            if pd.notnull(val):
                try:
                    return float(val)
                except Exception:
                    return None
        # orientacao B: rodadas no index, times nas colunas
        if (coluna_rodada in idx_row_map) and (nome_key in col_team_map):
            val = df_pontuacoes_times.at[idx_row_map[coluna_rodada], col_team_map[nome_key]]
            if pd.notnull(val):
                try:
                    return float(val)
                except Exception:
                    return None
        return None

    # --- 1.1) (NOVO) Se não for informado, inferir 'encerradas' pelo DF de pontuações ---
    if rodadas_encerradas is None:
        rodadas_encerradas = set()
        # usa as rodadas que aparecem no CSV de confrontos, convertidas p/ int
        candidatas = pd.to_numeric(df_rodadas["Rodada"], errors="coerce").dropna().astype(int).unique()
        for n in candidatas:
            if rodada_parcial is not None and n == rodada_parcial:
                continue  # nunca trate a parcial como encerrada

            col = f"Rodada {n}"
            if col not in df_pontuacoes_times.columns:
                continue

            serie = pd.to_numeric(df_pontuacoes_times[col], errors="coerce")
            num_validos = serie.notna().sum()
            tem_algum_ponto = (serie.fillna(0) > 0).any()

            # "Encerrada" = tem pelo menos 2 placares válidos e alguém > 0
            if num_validos >= 2 and tem_algum_ponto:
                rodadas_encerradas.add(n)

    # Sanity: se por engano passou a parcial dentro das encerradas, remove
    if rodada_parcial is not None and rodada_parcial in rodadas_encerradas:
        rodadas_encerradas.discard(rodada_parcial)

    # --- 2) Processa confrontos conforme status da rodada ---
    for _, confronto in df_rodadas.iterrows():
        try:
            rodada = int(confronto["Rodada"])
        except Exception:
            try:
                rodada = int(str(confronto["Rodada"]).strip())
            except Exception:
                continue

        mandante = str(confronto["Mandante_Nome"]).strip()
        visitante = str(confronto["Visitante_Nome"]).strip()
        grupo = str(confronto["Grupo"]).strip()
        coluna_rodada = f"Rodada {rodada}"

        # Rodada não elegível? (nem encerrada, nem a parcial atual) → ignora
        if (rodada not in rodadas_encerradas) and (rodada != rodada_parcial):
            continue

        pm = safe_pontuacao(mandante, coluna_rodada)
        pv = safe_pontuacao(visitante, coluna_rodada)

        # Se apenas um time tem pontuacao, assume 0 para o adversario (time nao escalou)
        if pm is None and pv is not None:
            pm = 0.0
        if pv is None and pm is not None:
            pv = 0.0

        # Regra para rodada PARCIAL: só conta se ambos têm número e pelo menos UM > 0
        if rodada == rodada_parcial:
            if pm is None and pv is None:
                continue
            if (pm == 0.0) and (pv == 0.0):
                continue

        # Regra para rodada ENCERRADA: exige números válidos para ambos
        if rodada in rodadas_encerradas:
            if pm is None and pv is None:
                continue
            if pm is None:
                pm = 0.0
            if pv is None:
                pv = 0.0

        # --- acumula totais ---
        if pm is None or pv is None:
            continue
        estatisticas[grupo][mandante]["Total_Cartola"] += pm
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pv
        estatisticas[grupo][visitante]["Total_Cartola"] += pv
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pm

        # --- pontos/vit/emp/der ---
        if pm > pv:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pm < pv:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # --- 3) Monta DataFrame final ---
    frames = []
    for grupo, times in estatisticas.items():
        if not times:
            continue
        df_g = pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
        })
        df_g["Saldo Cartola"] = df_g["Total Cartola"] - df_g["Cartola Sofrido"]
        frames.append(df_g)

    if not frames:
        cols = ["Grupo","Nome do Time","Pontos","Vitórias","Empates","Derrotas",
                "Total Cartola","Cartola Sofrido","Saldo Cartola","Posição"]
        return pd.DataFrame(columns=cols), {}

    df_resultado = pd.concat(frames, ignore_index=True).sort_values(
        by=["Grupo", "Pontos", "Total Cartola", "Saldo Cartola", "Nome do Time"],
        ascending=[True, False, False, False, True]
    ).reset_index(drop=True)

    df_resultado["Posição"] = df_resultado.groupby("Grupo").cumcount() + 1

    df_resultado_por_grupo = {
        g: df_resultado[df_resultado["Grupo"] == g].copy()
        for g in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo


In [13]:
# -------- Carregar confrontos da Pontos Corridos (turno selecionado) --------
ARQ_CONFRONTOS = Path("confrontos_pontos_corridos.csv")

# 1) Confrontos
if ARQ_CONFRONTOS.exists():
    df_confrontos = pd.read_csv(ARQ_CONFRONTOS)
    HAS_CONFRONTOS = not df_confrontos.empty
else:
    print("? confrontos_pontos_corridos.csv ainda n?o existe. Aguardando defini?o/cadastro dos confrontos.")
    df_confrontos = pd.DataFrame(columns=["Rodada", "Confronto", "Time A", "Time B"])
    HAS_CONFRONTOS = False

# 2) Renomeia colunas (se vierem como "Time A"/"Time B")
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3) Grupo fixo
if "Grupo" not in df_confrontos.columns:
    df_confrontos["Grupo"] = "Pontos Corridos"

# 4) Pontos Corridos do turno selecionado -> converter Rodada UI 1..19 para Cartola quando necessario
if "Rodada" in df_confrontos.columns:
    df_confrontos["Rodada"] = pd.to_numeric(df_confrontos["Rodada"], errors="coerce").fillna(0).astype(int)

if not df_confrontos.empty:
    if TURNO_SELECIONADO == 2 and df_confrontos["Rodada"].max() <= UI_FIM:
        df_confrontos["Rodada"] = df_confrontos["Rodada"] + TURNO_OFFSET
    HAS_CONFRONTOS = True

# ? Se n?o h? confrontos ainda, n?o calcula classifica?o
if df_confrontos.empty:
    print("? Sem confrontos cadastrados. Classifica?o ser? exibida quando os confrontos estiverem dispon?veis.")
    df_classificacao = pd.DataFrame()
    display(df_classificacao)
else:
    # 5) Status do mercado (para saber se a rodada corrente esta em andamento)
    def http_status_e_rodada():
        r = requests.get("https://api.cartola.globo.com/mercado/status", timeout=20)
        r.raise_for_status()
        d = r.json()
        return int(d.get("status_mercado", 0)), int(d.get("rodada_atual", 0))

    status_http, rodada_http = http_status_e_rodada()   # status 2 = em andamento
    rodada_http = int(rodada_http or TURNO_INICIO)

    # 6) Parametros para a classificacao
    rodada_parcial = rodada_http if (status_http == 2 and TURNO_INICIO <= rodada_http <= TURNO_FIM) else None
    if rodada_parcial:
        rodadas_encerradas = set(range(TURNO_INICIO, rodada_parcial))  # fim exclusivo
    else:
        rodadas_encerradas = set(range(TURNO_INICIO, min(rodada_http, TURNO_FIM) + 1))  # fim inclusivo

    print(f"[DEBUG] status={status_http} rodada_http={rodada_http} encerradas={sorted(rodadas_encerradas)} parcial={rodada_parcial}")

    # 7) Alinha nomes do DF de pontuacoes
    df_pontuacoes.index = df_pontuacoes.index.astype(str).str.strip()

    # 8) Classificacao
    df_classificacao, _ = classificacao_por_grupo(
        df_rodadas=df_confrontos,
        df_pontuacoes=df_pontuacoes,
        rodadas_encerradas=rodadas_encerradas,
        rodada_parcial=rodada_parcial,
    )

    # 9) Visualiza
    display(df_classificacao.head(30))


[DEBUG] status=2 rodada_http=1 encerradas=[] parcial=1


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
0,Pontos Corridos,Esquadrão Gazembrino,3,1,0,0,83.20,45.59,37.61,1
1,Pontos Corridos,GaúchoDaFronteira F.C,3,1,0,0,67.66,18.49,49.17,2
2,Pontos Corridos,SC 100 Sono,3,1,0,0,67.16,54.05,13.11,3
3,Pontos Corridos,Texas Club 2026,3,1,0,0,63.76,57.26,6.50,4
4,Pontos Corridos,bugredasmissões,3,1,0,0,62.76,56.69,6.07,5
5,Pontos Corridos,Doug Leal F.C,3,1,0,0,62.07,0.00,62.07,6
6,Pontos Corridos,lsauer fc,3,1,0,0,58.26,43.56,14.70,7
7,Pontos Corridos,Pontaç0 F.C.,3,1,0,0,56.86,43.96,12.90,8
8,Pontos Corridos,Medonho´s F.C.,3,1,0,0,55.66,42.06,13.60,9
9,Pontos Corridos,FBC Colorado,3,1,0,0,49.36,44.65,4.71,10


In [14]:

# ✅ Só faz sentido validar se temos confrontos e uma classificação com a coluna esperada
if df_confrontos is None or df_confrontos.empty:
    print("⏳ Sem confrontos cadastrados. Pulando validação de times.")
elif df_classificacao is None or df_classificacao.empty:
    print("⏳ Classificação ainda não gerada. Pulando validação de times.")
elif "Nome do Time" not in df_classificacao.columns:
    print(f"⚠️ Coluna 'Nome do Time' não encontrada na classificação. Colunas atuais: {list(df_classificacao.columns)}")
    print("Pulando validação de times.")
else:
    # 1. Times únicos nos confrontos
    cols_cf = [c for c in ["Mandante_Nome", "Visitante_Nome"] if c in df_confrontos.columns]
    if len(cols_cf) < 2:
        raise KeyError(f"Confrontos não têm as colunas esperadas. Colunas atuais: {list(df_confrontos.columns)}")

    times_confrontos = pd.unique(df_confrontos[cols_cf].values.ravel())

    # 2. Times únicos na classificação final
    times_classificados = df_classificacao["Nome do Time"].astype(str).str.strip().unique()

    # 3. Ver quem está nos confrontos mas não foi classificado
    faltando = set(map(str, times_confrontos)) - set(map(str, times_classificados))

    print("❌ Times que estão nos confrontos, mas faltam na classificação:")
    print(faltando)


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [15]:
# === Pontos Corridos — Exportadores (fail-safe) ===
import pandas as pd, json
from pathlib import Path

OUT_JS = Path("classificacao_pontos_corridos.js")
OUT_CSV = Path("classificacao_pontos_corridos.csv")

# Se ainda não há classificação válida, gera JS vazio e sai
if "df_classificacao" not in globals() or df_classificacao is None or df_classificacao.empty:
    classificacao_por_grupo = {"Pontos Corridos": []}
    with open(OUT_JS, "w", encoding="utf-8") as f:
        f.write("const classificacaoSerieA = ")
        json.dump(classificacao_por_grupo, f, ensure_ascii=False, indent=2)
        f.write(";")
    print("⚠️ df_classificacao vazio. Gerado classificacao_pontos_corridos.js vazio (Pontos Corridos: []).")
else:
    # tenta garantir que temos as colunas mínimas antes de exportar
    col_min = {"Grupo", "Nome do Time"}
    if not col_min.issubset(set(df_classificacao.columns)):
        print(f"⚠️ df_classificacao sem colunas mínimas {col_min}. Colunas atuais: {list(df_classificacao.columns)}")
        classificacao_por_grupo = {"Pontos Corridos": []}
        with open(OUT_JS, "w", encoding="utf-8") as f:
            f.write("const classificacaoSerieA = ")
            json.dump(classificacao_por_grupo, f, ensure_ascii=False, indent=2)
            f.write(";")
        print("✅ classificacao_pontos_corridos.js (vazio por falta de colunas)")
    else:
        # mantém seu fluxo original, só tornando robusto
        df_classificacao.to_csv(OUT_CSV, index=False)
        dfc = pd.read_csv(OUT_CSV)

        # renomeia só o que existir (evita KeyError)
        rename_map = {
            "Grupo": "grupo",
            "Nome do Time": "nome",
            "Pontos": "pontos",
            "Vitórias": "vitorias",
            "Empates": "empates",
            "Derrotas": "derrotas",
            "Total Cartola": "totalCartola",
            "Cartola Sofrido": "cartolaSofrido",
            "Saldo Cartola": "saldoCartola",
            "Posição": "posicao",
        }
        rename_map = {k: v for k, v in rename_map.items() if k in dfc.columns}
        dfc.rename(columns=rename_map, inplace=True)

        # garante que existe coluna 'grupo'
        if "grupo" not in dfc.columns:
            dfc["grupo"] = "Pontos Corridos"

        classificacao_por_grupo = {}
        for grupo, dados in dfc.groupby("grupo"):
            if "grupo" in dados.columns:
                dados = dados.drop(columns="grupo")
            classificacao_por_grupo[grupo] = dados.to_dict(orient="records")

        with open(OUT_JS, "w", encoding="utf-8") as f:
            f.write("const classificacaoSerieA = ")
            json.dump(classificacao_por_grupo, f, ensure_ascii=False, indent=2)
            f.write(";")
        print("✅ classificacao_pontos_corridos.js")


✅ classificacao_pontos_corridos.js


In [16]:
# -------------------------------------------
# 2) CONFRONTOS -> confrontos_pontos_corridos.js
# -------------------------------------------
if not ("df_confrontos" in globals()) or df_confrontos is None or df_confrontos.empty or not HAS_CONFRONTOS:
    print("? Sem confrontos. Gerando confrontos_pontos_corridos.js vazio.")
    df_confrontos_js = pd.DataFrame()
    with open("confrontos_pontos_corridos.js", "w", encoding="utf-8") as f:
        f.write("const confrontosFase1 = ")
        json.dump([], f, ensure_ascii=False, indent=2)
        f.write(";")
    print("? confrontos_pontos_corridos.js (vazio)")
else:
    df_confrontos_js = df_confrontos.copy()
    df_confrontos_js.columns = df_confrontos_js.columns.str.strip()
    df_confrontos_js.rename(columns={
        "Rodada": "rodada",
        "Confronto": "confronto",
        "Time A": "mandante_nome",
        "Time B": "visitante_nome",
        "Mandante_Nome": "mandante_nome",
        "Visitante_Nome": "visitante_nome",
        "ID A": "mandante_id",
        "ID B": "visitante_id",
    }, inplace=True)
    df_confrontos_js["grupo"] = "Pontos Corridos"

    # Normaliza rodada para UI (1..19) no JS
    if "rodada" in df_confrontos_js.columns:
        df_confrontos_js["rodada"] = pd.to_numeric(df_confrontos_js["rodada"], errors="coerce").fillna(0).astype(int)
        if TURNO_SELECIONADO == 2 and df_confrontos_js["rodada"].max() > UI_FIM:
            df_confrontos_js["rodada"] = df_confrontos_js["rodada"].map(rodada_cartola_para_ui)

    confrontos_formatado = []
    for _, row in df_confrontos_js.iterrows():
        confrontos_formatado.append({
            "rodada": int(row.get("rodada", 0) or 0),
            "confronto": int(row.get("confronto", 0) or 0),
            "grupo": row.get("grupo", "Pontos Corridos"),
            "mandante": {"id": row.get("mandante_id"), "nome": row.get("mandante_nome")},
            "visitante": {"id": row.get("visitante_id"), "nome": row.get("visitante_nome")},
        })

    with open("confrontos_pontos_corridos.js", "w", encoding="utf-8") as f:
        f.write("const confrontosFase1 = ")
        json.dump(confrontos_formatado, f, ensure_ascii=False, indent=2)
        f.write(";")
    print("? confrontos_pontos_corridos.js")


? confrontos_pontos_corridos.js


In [17]:
# -------------------------------------------
# 3) RESULTADOS (com parciais) -> resultados_pontos_corridos.js
# -------------------------------------------
# df_pontuacoes: index = nome do time, colunas = 'Rodada {TURNO_INICIO}'..'Rodada {TURNO_FIM}' (ou UI 1..19)

# normaliza df_pontuacoes
dfp = df_pontuacoes.copy()
dfp.index = dfp.index.astype(str).str.strip()
dfp.columns = [str(c) for c in dfp.columns]
dfp = dfp.apply(pd.to_numeric, errors="coerce")

if not ("df_confrontos_js" in globals()) or df_confrontos_js is None or df_confrontos_js.empty:
    print("? Sem confrontos. Gerando resultados_pontos_corridos.js vazio.")
    df_resultados = pd.DataFrame()
    with open("resultados_pontos_corridos.js", "w", encoding="utf-8") as f:
        f.write("const resultadosFase1 = ")
        json.dump([], f, ensure_ascii=False, indent=2)
        f.write(";")
        meta = {
            "rodada_parcial": int(rodada_parcial) if ("rodada_parcial" in globals() and rodada_parcial is not None) else None,
            "parcial_disponivel": bool(("rodada_parcial" in globals()) and (rodada_parcial is not None)),
        }
        f.write("\nwindow.pontosCorridosMeta = " + json.dumps(meta, ensure_ascii=False) + ";")
    print("? resultados_pontos_corridos.js (vazio)")
else:
    def _get_ponto(team: str, rodada_ui: int):
        """Busca ponto do 'team' em 'rodada_ui' aceitando:
           - coluna 'Rodada {rodada_cartola}' (turno selecionado),
           - ou 'Rodada {rodada_ui}' (caso tenha sido salvo assim)."""
        rodada_cartola = rodada_ui_para_cartola(rodada_ui)
        col_cartola = f"Rodada {rodada_cartola}"
        col_ui = f"Rodada {rodada_ui}"
        v = None
        if team in dfp.index and col_cartola in dfp.columns:
            v = dfp.at[team, col_cartola]
        if (v is None or pd.isna(v)) and team in dfp.index and col_ui in dfp.columns:
            v = dfp.at[team, col_ui]
        if v is None or pd.isna(v):
            return None
        return float(v)

    def gerar_resultados_pontos_corridos(df_confrontos_renamed: pd.DataFrame) -> pd.DataFrame:
        out = []
        for _, row in df_confrontos_renamed.iterrows():
            rodada = int(row["rodada"])
            mandante = str(row["mandante_nome"]).strip()
            visitante = str(row["visitante_nome"]).strip()
            pm = _get_ponto(mandante, rodada)
            pv = _get_ponto(visitante, rodada)
            out.append({
                "grupo": row.get("grupo", "Pontos Corridos"),
                "rodada": rodada,
                "mandante": {"nome": mandante, "pontos": None if pm is None else round(pm, 2)},
                "visitante": {"nome": visitante, "pontos": None if pv is None else round(pv, 2)},
            })
        return pd.DataFrame(out)

    df_resultados = gerar_resultados_pontos_corridos(df_confrontos_js)

    with open("resultados_pontos_corridos.js", "w", encoding="utf-8") as f:
        f.write("const resultadosFase1 = ")
        json.dump(df_resultados.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
        f.write(";")
        meta = {
            "rodada_parcial": int(rodada_parcial) if ("rodada_parcial" in globals() and rodada_parcial is not None) else None,
            "parcial_disponivel": bool(("rodada_parcial" in globals()) and (rodada_parcial is not None)),
        }
        f.write("\nwindow.pontosCorridosMeta = " + json.dumps(meta, ensure_ascii=False) + ";")
    print("? resultados_pontos_corridos.js (com parciais quando houver)")

# -------------------------------------------
# 4) PONTUACOES POR RODADA (opcional, pro front) -> pontuacoes_por_rodada_pontos_corridos.js
#    Gera chaves 'Rodada 1..19' a partir do df_pontuacoes
# -------------------------------------------
def df_to_ui_rounds(dfp: pd.DataFrame):
    mapa = {}
    for team in dfp.index:
        inner = {}
        for ui in range(UI_INICIO, UI_FIM + 1):
            col_cartola = f"Rodada {rodada_ui_para_cartola(ui)}"
            col_ui = f"Rodada {ui}"
            v = None
            if col_cartola in dfp.columns:
                vv = dfp.at[team, col_cartola]
                if pd.notna(vv):
                    v = float(vv)
            if v is None and col_ui in dfp.columns:
                vv = dfp.at[team, col_ui]
                if pd.notna(vv):
                    v = float(vv)
            if v is not None:
                inner[f"Rodada {ui}"] = v
        if inner:
            mapa[team] = inner
    return mapa

with open("pontuacoes_por_rodada_pontos_corridos.js", "w", encoding="utf-8") as f:
    f.write("window.pontuacoesPorRodada = ")
    json.dump(df_to_ui_rounds(dfp), f, ensure_ascii=False, indent=2)
    f.write(";")
print("? pontuacoes_por_rodada_pontos_corridos.js")

print("? Pronto. Recarregue a pagina com os 4 arquivos gerados incluidos antes do 'scripts/pontos_corridos.js'.")


? resultados_pontos_corridos.js (com parciais quando houver)
? pontuacoes_por_rodada_pontos_corridos.js
? Pronto. Recarregue a pagina com os 4 arquivos gerados incluidos antes do 'scripts/pontos_corridos.js'.


In [18]:
# Carregar o arquivo CSV enviado
if "df_confrontos" in globals() and df_confrontos is not None and not df_confrontos.empty:
    display(df_confrontos.head())
else:
    print("? Sem confrontos para exibir (confrontos_pontos_corridos.csv ainda nao existe).")


,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,FBC Colorado,GE Bebum,5,6,Pontos Corridos
1,1,2,GaúchoDaFronteira F.C,Pepe Leal FC,7,14,Pontos Corridos
2,1,3,Grêmio_Campeão_LA_27,SC 100 Sono,9,16,Pontos Corridos
3,1,4,Medonho´s F.C.,C R Juvenal,12,2,Pontos Corridos
4,1,5,bugredasmissões,Arran Katoko FC,19,1,Pontos Corridos


In [19]:
print(TURNO_SELECIONADO, TURNO_INICIO, TURNO_FIM, TURNO_OFFSET)
print(df_confrontos['Rodada'].min(), df_confrontos['Rodada'].max())
print(list(df_pontuacoes.columns)[:5], list(df_pontuacoes.columns)[-5:])


1 1 19 0
1 2
['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5'] ['Rodada 15', 'Rodada 16', 'Rodada 17', 'Rodada 18', 'Rodada 19']
